## Initialize

In [78]:
#for initialize
#!pip install google-cloud-translate==2.0.1

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="your-api"
from google.cloud import translate_v2 as translate
import html
import pandas as pd


: 

In [80]:
train_dir = '/opt/ml/KSR/dataset/train/train.csv'
train_df = pd.read_csv(train_dir)
sample_context = list(train_df.head(5)['sentence'])
sample_context

['〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 '호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.',
 'K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.',
 '균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.',
 '1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.']

## Translation

* `google translation api` 사용 
* context에는 back-translation할 `sentence list`가, 
* lang은 back-translation할 target 언어를 적으면 된다.(eg : `en`, `ja`, `fr`)

In [81]:
def back_translation_google(context, lang):
    back_translation_list = []
    client = translate.Client()

    for sentence in context:
        result = client.translate(sentence, target_language=lang)
        result = client.translate(result['translatedText'], target_language='ko')
        backtranslation_text = html.unescape(result["translatedText"])
        back_translation_list.append(backtranslation_text)
    
    return back_translation_list

In [82]:
back_translation_google(sample_context, "en")

['"Something"은 조지 해리슨이 작곡하고 비틀즈가 1969년 앨범 Abbey Road에 포함시킨 노래입니다.',
 '호남을 기점으로 한 바른미래당, 대안신당, 민주평화당이 우여곡절 끝에 합류해 민생당(가칭)으로 거듭났다.',
 "K리그2에서 1위를 달리고 있는 광주FC는 지난 26일 대한프로축구연맹(KFA)으로부터 관중 유치와 마케팅에 탁월한 성과를 인정받아 '풀스타디움상'과 '플러스스타디움상'을 수상했다.",
 '10일 균일가격 생활용품 전문점 아성다이소는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 해피박스를 배송했다고 10일 밝혔다.',
 '그는 1967년 프로야구 드래프트에서 1순위로 요미우리 자이언츠에 지명되었을 때 등번호 8번을 배정받았다.']

In [83]:
back_translation_google(sample_context, "ja")

["<Something>은 조지 해리슨이 써 비틀즈가 1969년 앨범 'Abbey Road'에 담은 곡이다.",
 '호남이 기반의 올바른 미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.',
 "K리그 2에서 성적 1위를 달리고 있는 광주 FC는 26일 한국 프로축구 연맹에서 관객 유치 성과와 마케팅 성과를 인정받아 '풀 스타디움상'과 '플러스 스타디움상'을 수상했다.",
 '균일가 생활용품점(주)아성다이소(대표 박 정부)는 코로나 19바이러스로 어려움을 겪고 있는 대구광역시 행복함을 전달했다고 10일 밝혔다.',
 '1967년 프로야구 드래프트 1위로 요미우리 자이언츠에 입단해 등번호는 8번으로 할당됐다.']

## Back tanslation with masking entities

In [84]:
#translation test
text = "균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다."
front = "<span tanslate=\"no\">"
back = "</span>"

mask_symbol1 = '_M_A_S_K_1_'
mask_symbol2 = '_M_A_S_K_2_'

def masktoken(text, word1, word2):
    text = text.replace(word1, mask_symbol1)
    text = text.replace(word2, mask_symbol2)

    return text

def demasktoken(text, word1, word2):
    text = text.replace(mask_symbol1, word1)
    text = text.replace(mask_symbol2, word2)

    return text

def addtoken(text, word):
    start = text.find(word)
    end = start + len(word)
    print(start)
    text_len = end - start + 1
    text = text[:start] + front + text[start:]
    text = text[:start+len(front)+text_len] + back + text[start+len(front)+text_len:] 

    return text

# text = addtoken(text, "광주FC")
# token_text = addtoken(text, "한국프로축구연맹")
# print(token_text)
original_text = text
text = masktoken(text, "아성다이소", "박정부")
mask_text = text
client = translate.Client()
result = client.translate(text, target_language='en')
result = client.translate(result['translatedText'], target_language='ko')
backtranslation_text = html.unescape(result["translatedText"])
text = demasktoken(backtranslation_text, "아성다이소", "박정부")
back_trans_text = text

In [85]:
print("Original One : ", original_text)
print("MASKED One : ", mask_text)
print("BackTansl One : ", back_trans_text)

Original One :  균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
MASKED One :  균일가 생활용품점 (주)_M_A_S_K_1_(대표 _M_A_S_K_2_)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
BackTansl One :  흥공가생활용품점아성다이소(대표 박정부)은 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 해피박스를 배송했다고 10일 밝혔다.


## Apply back-translation into train.csv

In [86]:
mask_symbol1 = '_M_A_S_K_1_'
mask_symbol2 = '_M_A_S_K_2_'

def masktoken(text, word1, word2):
    text = text.replace(word1, mask_symbol1)
    text = text.replace(word2, mask_symbol2)

    return text

def demasktoken(text, word1, word2):
    text = text.replace(mask_symbol1, word1)
    text = text.replace(mask_symbol2, word2)

    return text

def back_translation_train(text, sub, obj, lang):
    '''
    text-> train_df['sentence'], sub -> train_df['subject_entity']['word'],
    obj -> train_df['object_entity']['word']
    '''
    client = translate.Client()
    text = masktoken(text, sub, obj)
    client = translate.Client()
    result = client.translate(text, target_language=lang)
    result = client.translate(result['translatedText'], target_language='ko')
    backtranslation_text = html.unescape(result["translatedText"])
    text = demasktoken(backtranslation_text, sub, obj)

    return text

In [90]:
from tqdm import tqdm
train_dir = '/opt/ml/KSR/dataset/train/train.csv'
train_df = pd.read_csv(train_dir)

lang = ['en']
tqdm.pandas()
for la in lang:
    new_coulmn = f'{la}_trans_sentence'
    train_df[new_coulmn] = train_df.progress_apply(lambda row : 
    back_translation_train(row.sentence, eval(row.subject_entity)['word'],
    eval(row.object_entity)['word'], la),axis=1)
    

train_df.to_csv("/opt/ml/KSR/dataset/train/en_train.csv",columns=['en_trans_sentence'], index=False)

100%|██████████| 30/30 [00:22<00:00,  1.35it/s]
